In [32]:
import os
from typing import Dict
from dotenv import load_dotenv
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.utils import formatdate
from agents import function_tool

load_dotenv(override=True)

smtp_configuration = {
    "server": os.getenv("SMTP_SERVER"),
    "port": os.getenv("SMTP_PORT"),
    "username": os.getenv("SMTP_USERNAME"),
    "password": os.getenv("SMTP_PASSWORD"),
    "from": os.getenv("SMTP_SENDER_EMAIL")
}


@function_tool
def send_html_email(subject: str, body: str) -> Dict[str, str]:
    """Send out a HTML email with the given subject and body to all sales prospects"""

    msg = MIMEMultipart()
    msg['From'] = smtp_configuration["from"]
    msg['To'] = "nobelnurulislam@gmail.com"
    msg['Subject'] = subject
    msg['Date'] = formatdate(localtime=True) 

    msg.attach(MIMEText(body, 'html'))

    with smtplib.SMTP_SSL(smtp_configuration["server"], smtp_configuration["port"]) as server:
            server.login(smtp_configuration["username"], smtp_configuration["password"])
            # Combine all recipients
            recipients = ["nobelnurulislam@gmail.com"]
            server.sendmail(smtp_configuration["from"], recipients, msg.as_string())
        
    return {"status": "success"}

In [33]:
from agents import Agent


instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini",
)
tool1 = sales_agent1.as_tool("sales_agent1", instructions1)

sales_agent2 = Agent(
    name="Humorous Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini",
)
tool2 = sales_agent2.as_tool("sales_agent2", instructions2)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini",
)
tool3 = sales_agent3.as_tool("sales_agent3", instructions3)

email_generator_tools = [tool1, tool2, tool3]


In [34]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

email_sending_tools = [subject_tool, html_tool, send_html_email]

instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

email_sender_agent = Agent(name="Email sender",
                            instructions= instructions,
                            tools=email_sending_tools, 
                            model="gpt-4o-mini")


In [35]:
from agents import Runner, trace


handoffs = [email_sender_agent]

sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(name="Sales Manager",
                        instructions=sales_manager_instructions,
                        tools=email_generator_tools,
                        handoffs=handoffs,
                        model="gpt-4o-mini")


message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)